In [21]:
!pip install -U llama-cpp-python

In [22]:
"""!pip install llama-cpp-python

from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="lmstudio-community/olmOCR-7B-0225-preview-GGUF",
	filename="olmOCR-7B-0225-preview-Q4_K_M.gguf",
)
"""

'!pip install llama-cpp-python\n\nfrom llama_cpp import Llama\n\nllm = Llama.from_pretrained(\n\trepo_id="lmstudio-community/olmOCR-7B-0225-preview-GGUF",\n\tfilename="olmOCR-7B-0225-preview-Q4_K_M.gguf",\n)\n'

In [ ]:
from llama_cpp import Llama

print("Loading GGUF model with the correct 'qwen' chat format...")
llm = Llama.from_pretrained(
    repo_id="lmstudio-community/olmOCR-7B-0225-preview-GGUF",
    filename="olmOCR-7B-0225-preview-Q4_K_M.gguf",
    chat_format="qwen",
    n_ctx=2048,
    verbose=False
)
print("Model loaded successfully.")

Loading GGUF model with the correct 'qwen' chat format...


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


Model loaded successfully.


In [24]:
"""llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": [
				{
					"type": "text",
					"text": "Describe this image in one sentence."
				},
				{
					"type": "image_url",
					"image_url": {
						"url": "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"
					}
				}
			]
		}
	]
)"""

'llm.create_chat_completion(\n\tmessages = [\n\t\t{\n\t\t\t"role": "user",\n\t\t\t"content": [\n\t\t\t\t{\n\t\t\t\t\t"type": "text",\n\t\t\t\t\t"text": "Describe this image in one sentence."\n\t\t\t\t},\n\t\t\t\t{\n\t\t\t\t\t"type": "image_url",\n\t\t\t\t\t"image_url": {\n\t\t\t\t\t\t"url": "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"\n\t\t\t\t\t}\n\t\t\t\t}\n\t\t\t]\n\t\t}\n\t]\n)'

In [ ]:
import os
image_folder_path = "/kaggle/input/gltnnn/Gltn"

# Let's list the files to confirm the images are there
try:
    file_list = os.listdir(image_folder_path)
    if len(file_list) > 0:
        print(f"Successfully found {len(file_list)} files in the directory.")
        print("First 5 files:", file_list[:5])
    else:
        print("The directory is empty. Please check the path and your upload.")
except FileNotFoundError:
    print(f"Error: The directory '{image_folder_path}' was not found. Please double-check your dataset path.")

Successfully found 11 files in the directory.
First 5 files: ['IMG_1325.JPG', 'IMG_1317.JPG', 'IMG_1320.JPG', 'IMG_1319.JPG', 'IMG_1321.JPG']


In [26]:
!pip install llama-cpp-python Pillow fpdf2

In [ ]:
import base64
from io import BytesIO
from PIL import Image
import os

def image_to_base64(image, format="JPEG"):
    """Converts a PIL Image to a base64 encoded string."""
    buffered = BytesIO()
    image.save(buffered, format=format)
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

image_folder = image_folder_path
all_files = os.listdir(image_folder)
image_files = sorted([
    os.path.join(image_folder, f) for f in all_files
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
])

if not image_files:
    print("CRITICAL ERROR: No image files found.")
else:
    print(f"Found {len(image_files)} images to process.")

processed_images = []

print("Starting OCR processing with the most direct prompt...")
for image_path in image_files:
    image = Image.open(image_path).convert("RGB")
    image_b64 = image_to_base64(image)

    # Use the simplest, most direct command possible.
    prompt_text = "ocr"

    response = llm.create_chat_completion(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}},
                    # Add the direct command as text.
                    {"type": "text", "text": prompt_text}
                ]
            }
        ]
    )
    decoded_output = response['choices'][0]['message']['content']

    processed_images.append({
        "image_path": image_path,
        "text": decoded_output or " "
    })
    print(f"Processed: {os.path.basename(image_path)}")

print("OCR processing complete.")

Found 11 images to process.
Starting OCR processing with the most direct prompt...
Processed: IMG_1317.JPG
Processed: IMG_1318.JPG
Processed: IMG_1319.JPG
Processed: IMG_1320.JPG
Processed: IMG_1321.JPG
Processed: IMG_1322.JPG
Processed: IMG_1323.JPG
Processed: IMG_1324.JPG
Processed: IMG_1325.JPG
Processed: IMG_1326.JPG
Processed: IMG_1327.JPG
OCR processing complete.


In [ ]:
from fpdf import FPDF
import os

output_pdf_path = "ocr_text_document.pdf"

class PDF(FPDF):
    def header(self): pass
    def footer(self): pass

pdf = PDF()
pdf.set_font("Arial", size=12)

if not processed_images:
    print("Cannot create PDF, no images were processed.")
else:
    print(f"Creating text-only PDF from {len(processed_images)} processed items...")

    for i, item in enumerate(processed_images):
        pdf.add_page()

        image_basename = os.path.basename(item["image_path"]) 
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(0, 10, f"--- Text from: {image_basename} ---", 0, 1, 'C')
        pdf.ln(5)

        pdf.set_font("Arial", size=12)
        ocr_text = item["text"]
        printable_text = ocr_text.encode('latin-1', 'replace').decode('latin-1')

        pdf.multi_cell(0, 5, txt=printable_text)

    pdf.output(output_pdf_path)
    print(f"Success! Text-only PDF created at: {output_pdf_path}")

Creating text-only PDF from 11 processed items...
Success! Text-only PDF created at: ocr_text_document.pdf


/tmp/ipykernel_36/3172557940.py:11: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
/tmp/ipykernel_36/3172557940.py:24: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", 'B', 14)
/tmp/ipykernel_36/3172557940.py:25: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"--- Text from: {image_basename} ---", 0, 1, 'C')
/tmp/ipykernel_36/3172557940.py:28: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
/tmp/ipykernel_36/3172557940.py:32: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(0, 5, txt=printable_text)
